<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/multi_modal/anthropic_multi_modal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Modal LLM using Anthropic model for image reasoning

Anthropic has recently released its latest Multi modal models: Claude 3 Opus, Claude 3 Sonnet.

1. Claude 3 Opus - claude-3-opus-20240229

2. Claude 3 Sonnet - claude-3-sonnet-20240229

In this notebook, we show how to use Anthropic MultiModal LLM class/abstraction for image understanding/reasoning.

We also show several functions we are now supporting for Anthropic MultiModal LLM:
* `complete` (both sync and async): for a single prompt and list of images
* `chat` (both sync and async): for multiple chat messages
* `stream complete` (both sync and async): for steaming output of complete
* `stream chat` (both sync and async): for steaming output of chat

In [ ]:
%pip install llama-index-multi-modal-llms-anthropic

In [ ]:
!pip install matplotlib

##  Use Anthropic to understand Images from URLs

In [ ]:
import os

os.environ[
    "ANTHROPIC_API_KEY"
] = "YOUR ANTROPIC API KEY"  # Your ANTHROPIC API key here

## Initialize `AnthropicMultiModal` and Load Images from URLs

## 

In [ ]:
from llama_index.multi_modal_llms.anthropic import AnthropicMultiModal

from llama_index.core.multi_modal_llms.generic_utils import load_image_urls


image_urls = [
    "https://res.cloudinary.com/hello-tickets/image/upload/c_limit,f_auto,q_auto,w_1920/v1640835927/o3pfl41q7m5bj8jardk0.jpg",
]

image_documents = load_image_urls(image_urls)

anthropic_mm_llm = AnthropicMultiModal(max_tokens=300)

In [ ]:
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt

img_response = requests.get(image_urls[0])
print(image_urls[0])
img = Image.open(BytesIO(img_response.content))
plt.imshow(img)

### Complete a prompt with a bunch of images

In [ ]:
complete_response = anthropic_mm_llm.complete(
    prompt="Describe the images as an alternative text",
    image_documents=image_documents,
)

In [ ]:
print(complete_response)

### Steam Complete a prompt with a bunch of images

In [ ]:
stream_complete_response = anthropic_mm_llm.stream_complete(
    prompt="give me more context for this image",
    image_documents=image_documents,
)

In [ ]:
for r in stream_complete_response:
    print(r.delta, end="")

In [ ]:
from llama_index.core.multi_modal_llms.openai_utils import (
    generate_openai_multi_modal_chat_message,
)

chat_msg_1 = generate_openai_multi_modal_chat_message(
    prompt="Describe the images as an alternative text",
    role="user",
    image_documents=image_documents,
)

chat_msg_2 = generate_openai_multi_modal_chat_message(
    prompt="The image is a graph showing the surge in US mortgage rates. It is a visual representation of data, with a title at the top and labels for the x and y-axes. Unfortunately, without seeing the image, I cannot provide specific details about the data or the exact design of the graph.",
    role="assistant",
)

chat_msg_3 = generate_openai_multi_modal_chat_message(
    prompt="can I know more?",
    role="user",
)

chat_messages = [chat_msg_1, chat_msg_2, chat_msg_3]
chat_response = anthropic_mm_llm.chat(
    # prompt="Describe the images as an alternative text",
    messages=chat_messages,
)

In [ ]:
for msg in chat_messages:
    print(msg.role, msg.content)

In [ ]:
print(chat_response)

### Stream Chat through a list of chat messages

In [ ]:
stream_chat_response = anthropic_mm_llm.stream_chat(
    prompt="Describe the images as an alternative text",
    messages=chat_messages,
)

In [ ]:
for r in stream_chat_response:
    print(r.delta, end="")

### Async Complete

In [ ]:
response_acomplete = await anthropic_mm_llm.acomplete(
    prompt="Describe the images as an alternative text",
    image_documents=image_documents,
)

In [ ]:
print(response_acomplete)

### Async Steam Complete

In [ ]:
response_astream_complete = await anthropic_mm_llm.astream_complete(
    prompt="Describe the images as an alternative text",
    image_documents=image_documents,
)

In [ ]:
async for delta in response_astream_complete:
    print(delta.delta, end="")

### Async Chat

In [ ]:
achat_response = await anthropic_mm_llm.achat(
    messages=chat_messages,
)

In [ ]:
print(achat_response)

### Async stream Chat

In [ ]:
astream_chat_response = await anthropic_mm_llm.astream_chat(
    messages=chat_messages,
)

In [ ]:
async for delta in astream_chat_response:
    print(delta.delta, end="")

## Complete with Two images

In [ ]:
image_urls = [
    "https://www.visualcapitalist.com/wp-content/uploads/2023/10/US_Mortgage_Rate_Surge-Sept-11-1.jpg",
    "https://www.sportsnet.ca/wp-content/uploads/2023/11/CP1688996471-1040x572.jpg",
]

image_documents_1 = load_image_urls(image_urls)

response_multi = anthropic_mm_llm.complete(
    prompt="is there any relationship between those images?",
    image_documents=image_documents_1,
)
print(response_multi)

##  Use Anthropic Multi Model to understand images from local files

In [ ]:
from llama_index.core import SimpleDirectoryReader

# put your local directore here
image_documents = SimpleDirectoryReader("./data/").load_data()

response = anthropic_mm_llm.complete(
    prompt="Describe the images as an alternative text",
    image_documents=image_documents,
)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

img = Image.open("./data/1.jpg")
plt.imshow(img)

In [ ]:
print(response)